In [ ]:
import numpy as np
import struct 
import os
import plotly.graph_objects as go
import time

import lgchimera.general as general
from lgchimera.io import read_lidar_bin, read_gt
from lgchimera.kitti_util import process_kitti_gt, load_icp_results
from lgchimera.geom_util import euler_to_R
from lgchimera.pose_graph import PoseGraph

%load_ext autoreload
%autoreload 2

np.set_printoptions(suppress=True, precision=3)

Load ground truth

In [ ]:
kitti_seq = '0027'
start_idx = 0

gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

Load ICP results

In [ ]:
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'results', 'p2pl_icp')
lidar_Rs, lidar_ts, positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

FGO

In [ ]:
ATT_SIGMA = 1e-3 # [radians] 
ATT_INFO = np.eye(6)
ATT_INFO[:3,:3] *= 1e-6
ATT_INFO[3:,3:] *= 1e3
#ATT_INFO = (1/ATT_SIGMA**2) * np.eye(6)
#ATT_INFO *= 0.001
#ATT_INFO = np.zeros((6,6))

# Lidar information: fixed vs adaptive
LIDAR_INFO_FIXED = True
LIDAR_INFO = np.eye(6)
#LIDAR_INFO[:3,:3] *= 10 

ATT_RATE = 10  # Ratio of lidar to attitude measurements

In [ ]:
np.random.seed(0)

g = PoseGraph()
N_nodes = len(gt_enu)
window_size = 100

# Add initial node
R_abs = gt_Rs[0].copy()
t_abs = gt_enu[0].copy()
g.add_node(1, (R_abs, t_abs))

# Simulate attitude measurement
att_meas = gt_attitudes[0] #+ np.random.normal(0, ATT_SIGMA, 3) 
# Add attitude factor and edge
g.add_factor(1, (euler_to_R(att_meas), gt_enu[0]), information=ATT_INFO)

graph_positions = []
att_measurements = []

# For each new frame
for i in range(1,N_nodes):
    start_time = time.time()

    att_meas = gt_attitudes[i]
    R_att = euler_to_R(att_meas)

    # Get LiDAR odometry
    R_hat = np.array(lidar_Rs[i-1])
    t_hat = np.array(lidar_ts[i-1])

    # Initialize new node with LiDAR odometry estimate
    #R_abs = R_hat @ R_abs
    R_abs = R_att
    t_abs += R_abs @ t_hat
    g.add_node(i+1, (R_abs, t_abs))

    # Add LiDAR odometry edge
    if LIDAR_INFO_FIXED:
        lidar_information = LIDAR_INFO
    else:
        lidar_information = np.linalg.inv(lidar_covariances[i-1])
    g.add_edge([i, i+1], (R_hat, t_hat), information=lidar_information)

    if i % ATT_RATE == 0:
        # Simulate GPS measurement
         #+ np.random.normal(0, ATT_SIGMA, 3) 
        att_measurements.append(att_meas)

        # Add GPS factor and edge
        g.add_factor(i+1, (R_att, t_abs), information=ATT_INFO)

        # Trim to window size
        graph_size = max([v.id for v in g.graph._vertices])
        if graph_size > window_size:
            # Save pose before trimming
            graph_positions.append(g.get_positions()[:ATT_RATE])
            g.trim_window(n=ATT_RATE)
        
        # Optimize
        g.optimize()

        # Update latest pose
        R_abs, t_abs = g.get_poses()[-1]

        # Time each iteration
        print(i, "/", N_nodes, ": t=", time.time() - start_time, "s")

graph_positions = np.array(graph_positions).reshape(-1,3)
att_measurements = np.array(att_measurements)

In [ ]:
fgo_traj = go.Scatter(x=graph_positions[:,0], y=graph_positions[:,1], hovertext=np.arange(N_nodes), name='FGO trajectory')
lidar_traj = go.Scatter(x=positions[:,0], y=positions[:,1], hovertext=np.arange(N_nodes), name='Lidar trajectory')
gt_traj = go.Scatter(x=gt_enu[:N_nodes-window_size,0], y=gt_enu[:N_nodes-window_size,1], hovertext=np.arange(N_nodes), name='Ground-truth')
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
fig = go.Figure(data=[gt_traj, fgo_traj, lidar_traj, start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.75, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()